In [2]:
from tensorflow.python.keras import testing_utils
import warnings
warnings.filterwarnings('ignore')
from tensorflow.python import keras
import numpy as np
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers import recurrent_v2 as rnn
from tensorflow.python.framework import dtypes

/Users/rdua/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/rdua/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/rdua/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/rdua/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

In [3]:
batch = 100
timestep = 4
input_shape = 10
output_shape = 8
rnn_state_size = 8
epoch = 1

In [4]:
(x_train, y_train), _ = testing_utils.get_test_data(
        train_samples=batch,
        test_samples=0,
        input_shape=(timestep, input_shape),
        num_classes=output_shape)
y_train = np_utils.to_categorical(y_train, output_shape)

layer = rnn.GRU(rnn_state_size)

inputs = keras.layers.Input(
        shape=[timestep, input_shape], dtype=dtypes.float32)

outputs = layer(inputs)
    

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
len(x_train)

100

In [6]:
x_train[0]

array([[10.646834  , 14.099728  ,  0.6389914 ,  4.1535053 ,  0.14205575,
         4.04722   ,  8.726812  ,  1.094141  , 12.329019  , 10.609381  ],
       [ 0.99760777, 10.47655   , -0.4334075 ,  7.602825  , 10.024745  ,
        14.202351  ,  3.227591  ,  6.4155073 , 15.744299  ,  8.754451  ],
       [15.713361  ,  5.7794647 , 16.228895  , 11.079668  , 13.306903  ,
         3.5606604 ,  0.04428385,  5.8361025 ,  5.9516563 , 10.584612  ],
       [11.179455  ,  9.463918  ,  7.4499884 ,  1.6617529 , 13.601395  ,
        11.684782  ,  7.742146  ,  3.3160636 ,  7.0172    , 11.70673   ]],
      dtype=float32)

In [7]:
model = keras.models.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4, 10)]           0         
_________________________________________________________________
gru (GRU)                    (None, 8)                 480       
Total params: 480
Trainable params: 480
Non-trainable params: 0
_________________________________________________________________


In [9]:
len(x_train)

100

In [10]:

model.compile('rmsprop', loss='mse')
model.fit(x_train, y_train, epochs=epoch)
model.evaluate(x_train, y_train)
predictions = model.predict(x_train)

100/100 [==============================] - 1s 14ms/sample - loss: 0.7591


### Dynamic GRU

In [16]:
from tensorflow.python.training import gradient_descent

num_samples = 2
timesteps = 3
embedding_dim = 4
units = 2
layer = rnn.GRU(units, input_shape=(None, embedding_dim))
model = keras.models.Sequential()
model.add(layer)
model.compile(gradient_descent.GradientDescentOptimizer(0.001), 'mse')
x = np.random.random((num_samples, timesteps, embedding_dim))
y = np.random.random((num_samples, units))
model.train_on_batch(x, y)

0.15944166

## Stacking GRUs

In [18]:
inputs = np.random.random((2, 3, 4))
targets = np.abs(np.random.random((2, 3, 5)))
targets /= targets.sum(axis=-1, keepdims=True)
model = keras.models.Sequential()
model.add(rnn.GRU(10, return_sequences=True, unroll=False))
model.add(rnn.GRU(5, return_sequences=True, unroll=False))
model.compile(
        loss='categorical_crossentropy',
        optimizer=gradient_descent.GradientDescentOptimizer(0.01))
model.fit(inputs, targets, epochs=1, batch_size=2, verbose=1)

2/2 [==============================] - 5s 2s/sample - loss: 3.8388


## GRU with Masking

In [19]:
 layer_class = rnn.GRU
inputs = np.random.random((2, 3, 4))
targets = np.abs(np.random.random((2, 3, 5)))
targets /= targets.sum(axis=-1, keepdims=True)
model = keras.models.Sequential()
model.add(keras.layers.Masking(input_shape=(3, 4)))
model.add(layer_class(units=5, return_sequences=True, unroll=False))
model.compile(loss='categorical_crossentropy',
                  optimizer=gradient_descent.GradientDescentOptimizer(0.001))
model.fit(inputs, targets, epochs=1, batch_size=2, verbose=1)

2/2 [==============================] - 4s 2s/sample - loss: 7.7460


## Return state == True

In [20]:
x = np.random.random((2, 3, 4))
y = np.abs(np.random.random((2, 5)))
s = np.abs(np.random.random((2, 5)))
inputs = keras.layers.Input(
        shape=[3, 4], dtype=dtypes.float32)
masked = keras.layers.Masking()(inputs)
outputs, states = layer_class(units=5, return_state=True)(masked)

model = keras.models.Model(inputs, [outputs, states])
model.compile(loss='categorical_crossentropy',
                  optimizer=gradient_descent.GradientDescentOptimizer(0.001))
model.fit(x, [y, s], epochs=1, batch_size=2, verbose=1)

2/2 [==============================] - 3s 2s/sample - loss: 31.5371 - gru_9_loss: 15.8029 - gru_9_1_loss: 15.7341


## GRU with Constraints


In [33]:
from tensorflow.python.keras import keras_parameterized
embedding_dim = 4
layer_class = rnn.GRU
k_constraint = keras.constraints.max_norm(0.01)
r_constraint = keras.constraints.max_norm(0.01)
b_constraint = keras.constraints.max_norm(0.01)
layer = layer_class(
        5,
        return_sequences=False,
        weights=None,
        input_shape=(None, embedding_dim),
        kernel_constraint=k_constraint,
        recurrent_constraint=r_constraint,
        bias_constraint=b_constraint)
layer.build((None, None, embedding_dim))


In [34]:
layer